## Attempt -1
    This is me trying to classify using tensor flow . 
>Error : The use of 'string' is not permitted in a model and is to be converted to an integer

Do read README.md/ipynb file 

In [190]:
import tensorflow as tf
import pandas as pd
from pathlib import Path
import numpy as np

In [191]:
data_folder = Path("Test_ag.csv")

In [192]:
with open(data_folder) as handler:
    df = pd.read_csv(handler,index_col=False)
    df.pop('author')
    df = df.where(df.notnull(), 'U')

In [193]:
df.isnull().sum()

title      0
year       0
authors    0
jconf      0
pid        0
label      0
dtype: int64

In [194]:
# df2 = df.columns.values.tolist()
CSV_COLUMN_NAMES = ['title', 'year', 'authors', 'jconf', 'pid', 'label']
# SPECIES = [i for i in  range(df.loc[:,"label"].max()+1)]
LABELS = [0, 1, 2, 3, 4, 5, 6, 7, 8]
LABELS,CSV_COLUMN_NAMES

([0, 1, 2, 3, 4, 5, 6, 7, 8],
 ['title', 'year', 'authors', 'jconf', 'pid', 'label'])

In [195]:
df = df.sample(len(df))
length  = len(df)
length

36

In [196]:
train_len = int(0.8 * length)
train_len
test_len = int(0.1 * length)
test_len

3

In [197]:
train = df[0: train_len]
train_y = train.pop('label')
test = df[train_len: train_len+test_len]
test_y= test.pop('label')
valid = df[train_len+test_len:]
train_set.shape

(28, 6)

In [198]:
# train.to_csv('train.csv', header=True, index=False, float_format='%.4f')
# valid.to_csv('valid.csv', header=True, index=False, float_format='%.4f')

In [199]:
# train = pd.read_csv('train.csv')
# valid = pd.read_csv('valid.csv')

In [200]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [215]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.categorical_column_with_hash_bucket('key',hash_bucket_size=1000))
print(my_feature_columns)

[HashedCategoricalColumn(key='key', hash_bucket_size=1000, dtype=tf.string), HashedCategoricalColumn(key='key', hash_bucket_size=1000, dtype=tf.string), HashedCategoricalColumn(key='key', hash_bucket_size=1000, dtype=tf.string), HashedCategoricalColumn(key='key', hash_bucket_size=1000, dtype=tf.string), HashedCategoricalColumn(key='key', hash_bucket_size=1000, dtype=tf.string)]


In [216]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[50, 50],
    # The model must choose between 3 classes.
    n_classes=len(LABELS)
#     ,activation_fn='relu',batch_norm=50
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\KHADGA~1\\AppData\\Local\\Temp\\tmp3eoo6j5g', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [217]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)
# We include a lambda to avoid creating an inner function previously

INFO:tensorflow:Calling model_fn.


ValueError: Duplicate feature column name found for columns: HashedCategoricalColumn(key='key', hash_bucket_size=1000, dtype=tf.string) and HashedCategoricalColumn(key='key', hash_bucket_size=1000, dtype=tf.string). This usually means that these columns refer to same base feature. Either one must be discarded or a duplicated but renamed item must be inserted in features dict.